**AUTHOR: RAIHAN SALMAN BAEHAQI (1103220180)**

**PART I** 

**The Fundamentals of Machine Learning** 

---

**CHAPTER 9 - Unsupervised Learning Techniques** 

---

Although most ML applications today are based on supervised learning, the vast majority of available data is unlabeled: we have input features X but not labels y. Computer scientist Yann LeCun famously said: "if intelligence was a cake, unsupervised learning would be the cake, supervised learning would be the icing on the cake, and reinforcement learning would be the cherry on the cake." 

Consider a manufacturing production line system taking thousands of pictures daily to detect defective items. Creating a labeled dataset requires human experts to manually label every picture—a long, costly, tedious task. This results in a small labeled dataset and disappointing classifier performance. Moreover, every product change requires restarting the entire labeling process. 

This chapter covers three main unsupervised learning tasks: 

**Clustering** - Groups similar instances together into clusters. Great for data analysis, customer segmentation, recommender systems, search engines, image segmentation, semi-supervised learning, and dimensionality reduction. 

**Anomaly detection** - Learns what "normal" data looks like to detect abnormal instances, such as defective items on a production line or new trends in time series. 

**Density estimation** - Estimates the probability density function (PDF) of the random process that generated the dataset. Commonly used for anomaly detection (instances in low-density regions are likely anomalies) and for data analysis and visualization. 

---

## **Clustering** 

Clustering identifies similar instances and assigns them to clusters, or groups of similar instances. Unlike classification, clustering is unsupervised. 

**Figure 9-1. Classification (left) versus clustering (right)**   
![Figure9-1.jpg](./09.Chapter-09/Figure9-1.jpg) 

The iris dataset on the left is labeled (suitable for classification), while the right shows the same dataset without labels. Clustering algorithms easily detect the lower-left cluster, though it's not obvious the upper-right contains two distinct sub-clusters. Using all features, clustering algorithms like Gaussian mixture models identify the three clusters fairly well (only 5 out of 150 instances misclassified). 

**Applications:** Customer segmentation, data analysis, dimensionality reduction, anomaly detection, semi-supervised learning, search engines, and image segmentation. 

---

### **K-Means** 

**Figure 9-2. An unlabeled dataset composed of five blobs of instances**   
![Figure9-2.jpg](./09.Chapter-09/Figure9-2.jpg) 

The **K-Means algorithm** clusters datasets very quickly and efficiently, often in just a few iterations. Proposed by Stuart Lloyd at Bell Labs in 1957 as a technique for pulse-code modulation, it was only published outside the company in 1982. In 1965, Edward W. Forgy published virtually the same algorithm, so it's sometimes called Lloyd–Forgy.

In [ ]:
from sklearn.cluster import KMeans

k = 5
kmeans = KMeans(n_clusters=k)
y_pred = kmeans.fit_predict(X)

You must specify the number of clusters k. Each instance was assigned to one of five clusters. In clustering context, an instance's **label** is the index of the cluster assigned by the algorithm—not to be confused with class labels in classification.

In [ ]:
>>> y_pred
array([4, 0, 1, ..., 2, 1, 0], dtype=int32)
>>> y_pred is kmeans.labels_
True
>>> kmeans.cluster_centers_
array([[-2.80389616,  1.80117999],
       [ 0.20876306,  2.25551336],
       [-2.79290307,  2.79641063],
       [-1.46679593,  2.28585348],
       [-2.80037642,  1.30082566]])

You can easily assign new instances to the cluster whose centroid is closest:

In [ ]:
>>> X_new = np.array([[0, 2], [3, 2], [-3, 3], [-3, 2.5]])
>>> kmeans.predict(X_new)
array([1, 1, 2, 2], dtype=int32)

**Figure 9-3. K-Means decision boundaries (Voronoi tessellation)**   
![Figure9-3.jpg](./09.Chapter-09/Figure9-3.jpg) 

The decision boundaries form a **Voronoi tessellation**. Most instances were clearly assigned to the appropriate cluster, but a few near boundaries were probably mislabeled. K-Means doesn't behave well when blobs have very different diameters because all it cares about is distance to the centroid. 

Instead of assigning each instance to a single cluster (**hard clustering**), it can be useful to give each instance a score per cluster (**soft clustering**). The score can be the distance between the instance and centroid, or a similarity score (affinity) like the Gaussian RBF. 

The transform() method measures distance from each instance to every centroid:

In [ ]:
>>> kmeans.transform(X_new)
array([[2.81093633, 0.32995317, 2.9042344 , 1.49439034, 2.88633901],
       [5.80730058, 2.80290755, 5.84739223, 4.4759332 , 5.84236351],
       [1.21475352, 3.29399768, 0.29040966, 1.69136631, 1.71086031],
       [0.72581411, 3.21806371, 0.36159148, 1.54808703, 1.21567622]])

If you have a high-dimensional dataset and transform it this way, you end up with a k-dimensional dataset: this can be a very efficient nonlinear dimensionality reduction technique. 

#### **The K-Means Algorithm** 

The algorithm places centroids randomly, labels instances, updates centroids, and repeats until convergence. It's guaranteed to converge in a finite number of steps (usually quite small) because mean squared distance between instances and their closest centroid can only go down at each step. 

**Figure 9-4. The K-Means algorithm**   
![Figure9-4.jpg](./09.Chapter-09/Figure9-4.jpg) 

The figure shows centroids initialized randomly (top left), then instances labeled (top right), centroids updated (center left), instances relabeled (center right), and so on. In just three iterations, the algorithm reached clustering that seems close to optimal. 

**Computational Complexity:** The algorithm is generally linear with regard to number of instances m, number of clusters k, and number of dimensions n. However, this is only true when data has a clustering structure. In worst-case scenarios, complexity can increase exponentially with m. In practice, this rarely happens, and K-Means is generally one of the fastest clustering algorithms. 

Although guaranteed to converge, it may not converge to the right solution (may converge to a local optimum), depending on centroid initialization. 

**Figure 9-5. Suboptimal solutions due to unlucky centroid initializations**   
![Figure9-5.jpg](./09.Chapter-09/Figure9-5.jpg) 

#### **Centroid Initialization Methods** 

If you know approximately where centroids should be, set the init hyperparameter to a NumPy array containing the centroid list, and set n_init to 1:

In [ ]:
good_init = np.array([[-3, 3], [-3, 2], [-3, 1], [-1, 2], [0, 2]])
kmeans = KMeans(n_clusters=5, init=good_init, n_init=1)

Another solution is running the algorithm multiple times with different random initializations and keeping the best solution. The number of random initializations is controlled by n_init (default is 10), meaning the whole algorithm runs 10 times when you call fit(), and Scikit-Learn keeps the best solution. 

But how does it know which solution is best? It uses the model's **inertia**: the mean squared distance between each instance and its closest centroid. The KMeans class runs the algorithm n_init times and keeps the model with the lowest inertia. 

A model's inertia is accessible via the inertia_ instance variable:

In [ ]:
>>> kmeans.inertia_
211.59853725816856

The score() method returns the negative inertia. Why negative? Because a predictor's score() method must always respect Scikit-Learn's "greater is better" rule:

In [ ]:
>>> kmeans.score(X)
-211.59853725816856

**K-Means++** is an important improvement proposed in 2006 by David Arthur and Sergei Vassilvitskii. They introduced a smarter initialization step that tends to select centroids that are distant from one another, making K-Means much less likely to converge to a suboptimal solution. They showed the additional computation is well worth it because it drastically reduces the number of times the algorithm needs to be run. 

The KMeans class uses this initialization by default. To force it to use the original method (picking k instances randomly), set init="random". 

#### **Accelerated K-Means and Mini-batch K-Means** 

Another important improvement was proposed in 2003 by Charles Elkan. It considerably accelerates the algorithm by avoiding many unnecessary distance calculations by exploiting the **triangle inequality** and keeping track of lower and upper bounds for distances between instances and centroids. This is the default algorithm. To force the original algorithm, set algorithm="full". 

Another variant proposed in 2010 by David Sculley uses **mini-batches** instead of the full dataset at each iteration, moving centroids just slightly at each iteration. This speeds up the algorithm typically by a factor of three or four and makes it possible to cluster huge datasets that don't fit in memory. Scikit-Learn implements this in MiniBatchKMeans:

In [ ]:
from sklearn.cluster import MiniBatchKMeans

minibatch_kmeans = MiniBatchKMeans(n_clusters=5)
minibatch_kmeans.fit(X)

If the dataset doesn't fit in memory, the simplest option is to use the memmap class. Alternatively, pass one mini-batch at a time to partial_fit(), but this requires much more work since you'll need to perform multiple initializations and select the best one yourself. 

Although Mini-batch K-Means is much faster than regular K-Means, its inertia is generally slightly worse, especially as k increases. 

**Figure 9-6. Mini-batch K-Means has a higher inertia than K-Means (left) but it is much faster (right), especially as k increases**   
![Figure9-6.jpg](./09.Chapter-09/Figure9-6.jpg) 

#### **Finding the Optimal Number of Clusters** 

Setting k to the wrong value results in fairly bad models. 

**Figure 9-7. Bad choices for the number of clusters**   
![Figure9-7.jpg](./09.Chapter-09/Figure9-7.jpg) 

When k is too small, separate clusters get merged (left); when k is too large, some clusters get chopped into multiple pieces (right). 

Inertia is not a good performance metric when trying to choose k because it keeps getting lower as we increase k. Indeed, the more clusters there are, the closer each instance will be to its closest centroid, and therefore the lower the inertia will be. 

Let's plot inertia as a function of k: 

**Figure 9-8. When plotting the inertia as a function of the number of clusters k, the curve often contains an inflexion point called the "elbow"**   
![Figure9-8.jpg](./09.Chapter-09/Figure9-8.jpg) 

Inertia drops very quickly as we increase k up to 4, but then decreases much more slowly as we keep increasing k. This curve has roughly the shape of an arm, and there's an "elbow" at k = 4. This technique for choosing k is rather coarse. 

A more precise approach (but also more computationally expensive) is to use the **silhouette score**: the mean silhouette coefficient over all instances. An instance's silhouette coefficient equals (b – a) / max(a, b), where a is the mean distance to other instances in the same cluster (mean intra-cluster distance), and b is the mean nearest-cluster distance (mean distance to instances of the next closest cluster). 

The silhouette coefficient varies between –1 and +1. A coefficient close to +1 means the instance is well inside its own cluster and far from other clusters. A coefficient close to 0 means it's close to a cluster boundary. A coefficient close to –1 means the instance may have been assigned to the wrong cluster.

In [ ]:
>>> from sklearn.metrics import silhouette_score
>>> silhouette_score(X, kmeans.labels_)
0.655517642572828

**Figure 9-9. Selecting the number of clusters k using the silhouette score**   
![Figure9-9.jpg](./09.Chapter-09/Figure9-9.jpg) 

This visualization is much richer than the previous one: although it confirms k = 4 is a very good choice, it also underlines that k = 5 is quite good as well, and much better than k = 6 or 7. 

An even more informative visualization is obtained by plotting every instance's silhouette coefficient, sorted by cluster assignment and by coefficient value. This is called a **silhouette diagram**. 

**Figure 9-10. Analyzing the silhouette diagrams for various values of k**   
![Figure9-10.jpg](./09.Chapter-09/Figure9-10.jpg) 

Each diagram contains one knife shape per cluster. The shape's height indicates the number of instances the cluster contains, and its width represents the sorted silhouette coefficients of instances in the cluster (wider is better). The dashed line indicates the mean silhouette coefficient. 

When most instances in a cluster have a lower coefficient than this score (if many instances stop short of the dashed line, ending to its left), the cluster is rather bad since its instances are much too close to other clusters. 

---

### **Limits of K-Means** 

Despite its many merits—notably being fast and scalable—K-Means is not perfect. It's necessary to run the algorithm several times to avoid suboptimal solutions, plus you need to specify the number of clusters. Moreover, **K-Means does not behave very well when clusters have varying sizes, different densities, or nonspherical shapes**. 

**Figure 9-11. K-Means fails to cluster these ellipsoidal blobs properly**   
![Figure9-11.jpg](./09.Chapter-09/Figure9-11.jpg) 

The figure shows how K-Means clusters a dataset containing three ellipsoidal clusters of different dimensions, densities, and orientations. Neither solution is any good. The solution on the left is better, but it still chops off 25% of the middle cluster and assigns it to the cluster on the right. The solution on the right is terrible, even though its inertia is lower. So depending on the data, different clustering algorithms may perform better. On these types of elliptical clusters, Gaussian mixture models work great. 

**It is important to scale the input features before running K-Means**, or clusters may be very stretched and K-Means will perform poorly. Scaling features doesn't guarantee all clusters will be nice and spherical, but it generally improves things. 

---

### **Using Clustering for Image Segmentation** 

**Image segmentation** is the task of partitioning an image into multiple segments. In **semantic segmentation**, all pixels that are part of the same object type get assigned to the same segment. In **instance segmentation**, all pixels that are part of the same individual object are assigned to the same segment. 

The state of the art in semantic or instance segmentation today is achieved using complex architectures based on convolutional neural networks (see Chapter 14). Here, we'll do something much simpler: **color segmentation**. We'll simply assign pixels to the same segment if they have similar colors. In some applications, this may be sufficient.

In [ ]:
>>> from matplotlib.image import imread
>>> image = imread(os.path.join("images","unsupervised_learning","ladybug.png"))
>>> image.shape
(533, 800, 3)

The image is represented as a 3D array. The first dimension is the height (533 pixels), the second is the width (800 pixels), and the third is the number of color channels—in this case there are three: red, green, and blue (RGB). For each pixel there's a 3D vector containing red, green, and blue intensities (each between 0.0 and 1.0). Some images may have fewer channels (e.g., grayscale with one channel) or more (e.g., with an additional alpha channel for transparency, or satellite images with channels for many light frequencies like infrared).

In [ ]:
X = image.reshape(-1, 3)
kmeans = KMeans(n_clusters=8).fit(X)
segmented_img = kmeans.cluster_centers_[kmeans.labels_]
segmented_img = segmented_img.reshape(image.shape)

The code reshapes the array to get a long list of RGB colors, clusters these colors using K-Means, and for each pixel color (e.g., dark green), it looks for the mean color of that pixel's color cluster. Finally, it reshapes this long list of colors to get the same shape as the original image. 

**Figure 9-12. Image segmentation using K-Means with various numbers of color clusters**   
![Figure9-12.jpg](./09.Chapter-09/Figure9-12.jpg) 

You can experiment with various numbers of clusters. When using fewer than eight clusters, notice the ladybug's flashy red color fails to get its own cluster—it gets merged with colors from the environment. This is because **K-Means prefers clusters of similar sizes**. 

---

### **Using Clustering for Preprocessing** 

Clustering can be an efficient approach to dimensionality reduction, in particular as a preprocessing step before a supervised learning algorithm. For example, let's tackle the digits dataset (introduced in Chapter 3). This dataset comprises 1,797 grayscale 8×8 images representing digits 0 to 9. Let's train a Logistic Regression model on it:

In [ ]:
from sklearn.datasets import load_digits
X_digits, y_digits = load_digits(return_X_y=True)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits)

from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

>>> log_reg.score(X_test, y_test)
0.9688888888888889

The accuracy is 96.89%. Let's see if we can do better by using K-Means as a preprocessing step. We will create a pipeline that will first cluster the training set into 50 clusters and replace the images with their distances to these 50 clusters, then apply a Logistic Regression model:

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ("kmeans", KMeans(n_clusters=50)),
    ("log_reg", LogisticRegression()),
])
pipeline.fit(X_train, y_train)

>>> pipeline.score(X_test, y_test)
0.9777777777777777

How about that? We reduced the error rate by almost 30%! This is because the algorithm now has the distances to all 50 clusters as features, meaning it can measure similarity to each cluster. 

Let's search for the optimal number of clusters k using grid search:

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = dict(kmeans__n_clusters=range(2, 100))
grid_clf = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_clf.fit(X_train, y_train)

>>> grid_clf.best_params_
{'kmeans__n_clusters': 99}
>>> grid_clf.score(X_test, y_test)
0.9822222222222222

The error rate was reduced by over 34%! 

---

### **Using Clustering for Semi-Supervised Learning** 

Another use case for clustering is in semi-supervised learning, when we have plenty of unlabeled instances and very few labeled instances. Let's train a Logistic Regression model on a sample of 50 labeled instances from the digits dataset:

In [ ]:
n_labeled = 50
log_reg = LogisticRegression()
log_reg.fit(X_train[:n_labeled], y_train[:n_labeled])

>>> log_reg.score(X_test, y_test)
0.8333333333333334

The accuracy is only 83.3%. It should come as no surprise that this is much lower than earlier, when we trained the model on the full training set. Let's see how we can do better. First, let's cluster the training set into 50 clusters, then for each cluster let's find the image closest to the centroid. We will call these images the representative images:

In [ ]:
k = 50
kmeans = KMeans(n_clusters=k)
X_digits_dist = kmeans.fit_transform(X_train)
representative_digit_idx = np.argmin(X_digits_dist, axis=0)
X_representative_digits = X_train[representative_digit_idx]

**Figure 9-13. Fifty representative digit images (one per cluster)**   
![Figure9-13.jpg](./09.Chapter-09/Figure9-13.jpg) 

Now let's manually label these representative images:

In [ ]:
>>> y_representative_digits = np.array([4, 8, 0, 6, 8, 3, ..., 7, 6, 2, 3, 1, 1])

Now we have a dataset with just 50 labeled instances, but instead of being completely random instances, each of them is a representative image of its cluster. Let's see if the performance is any better:

In [ ]:
>>> log_reg = LogisticRegression()
>>> log_reg.fit(X_representative_digits, y_representative_digits)
>>> log_reg.score(X_test, y_test)
0.9222222222222223

Wow! We jumped from 83.3% accuracy to 92.2%, although we are still only training the model on 50 instances. Since it's often costly and painful to label instances, especially when it has to be done manually by experts, it's a good idea to make them label representative instances rather than just random instances. 

But perhaps we can go one step further: what if we propagated the labels to all the other instances in the same cluster?

In [ ]:
y_train_propagated = np.empty(len(X_train), dtype=np.int32)
for i in range(k):
    y_train_propagated[kmeans.labels_==i] = y_representative_digits[i]

>>> log_reg.fit(X_train, y_train_propagated)
>>> log_reg.score(X_test, y_test)
0.9333333333333333

We got a tiny little accuracy boost. Better than nothing, but we should probably propagate the labels only to the instances closest to the centroids (e.g., within the 20th percentile, which represents about 10% of the diameter of each cluster), since these are the instances that are most likely to be labeled correctly:

In [ ]:
percentile_closest = 20
X_cluster_dist = X_digits_dist[np.arange(len(X_train)), kmeans.labels_]
for i in range(k):
    in_cluster = (kmeans.labels_ == i)
    cluster_dist = X_cluster_dist[in_cluster]
    cutoff_distance = np.percentile(cluster_dist, percentile_closest)
    above_cutoff = (X_cluster_dist > cutoff_distance)
    X_cluster_dist[in_cluster & above_cutoff] = -1

partially_propagated = (X_cluster_dist != -1)
X_train_partially_propagated = X_train[partially_propagated]
y_train_partially_propagated = y_train_propagated[partially_propagated]

>>> log_reg = LogisticRegression()
>>> log_reg.fit(X_train_partially_propagated, y_train_partially_propagated)
>>> log_reg.score(X_test, y_test)
0.94

Nice! With just 50 labeled instances (just 5 examples per class on average), we got 94.0% performance, which is getting closer to the performance of Logistic Regression on the full training set (96.9%). 

#### **Active Learning** 

To continue improving the model, we could do several more iterations of **active learning**: 
1. Manually label the instances that the model is most uncertain about
2. Train a new model using these additional labels
3. Repeat until performance improvement stops 

This strategy is possible because most instances near decision boundaries or in low-density regions are typically among those for which the model is most uncertain. The most common active learning strategy is **uncertainty sampling**, where a human expert is asked to label the instances for which the model is most uncertain (e.g., when its estimated probability is lowest). Other strategies exist, such as selecting instances that would result in the largest model change, or that would result in the greatest reduction in the model's generalization error (measured using the remaining labeled instances). 

---

### **DBSCAN** 

**DBSCAN** (Density-Based Spatial Clustering of Applications with Noise) is another clustering algorithm. It defines clusters as continuous regions of high density. Here's how it works: for each instance, the algorithm counts how many instances are located within a small distance ε (epsilon) from it. This region is called the instance's **ε-neighborhood**. If an instance has at least min_samples instances in its ε-neighborhood (including itself), then it is considered a **core instance**. All instances in the neighborhood of a core instance belong to the same cluster. This includes core instances in the neighborhood, forming a long sequence of neighboring core instances. Any instance that is not a core instance and does not have one in its neighborhood is considered an **anomaly**. 

This algorithm works well if all the clusters are dense enough, and if they are well separated by low-density regions. The following code applies DBSCAN to the moons dataset:

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=1000, noise=0.05)
dbscan = DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)

All instances were assigned to a cluster (there are no anomalies in this dataset). Let's look at the labels assigned:

In [ ]:
>>> dbscan.labels_
array([ 0,  2, -1, -1,  1,  0,  0,  0, ...,  3,  2,  3,  3,  4,  2,  6,  3])

Notice that some instances have a cluster label of –1: this means that they are considered as anomalies by the algorithm. The core instances are in the components_ instance variable, and the indices of their corresponding labels are in the core_sample_indices_ instance variable. 

**Figure 9-14. DBSCAN clustering using two different neighborhood radiuses**   
![Figure9-14.jpg](./09.Chapter-09/Figure9-14.jpg) 

This figure shows what happens when you use two different values for the eps hyperparameter. When it is set too small, many instances end up being considered anomalies (–1). When it is set too large, distant clusters start to merge. The DBSCAN class does not have a predict() method, although it has a fit_predict() method. In other words, it cannot predict which cluster a new instance belongs to. This is a fundamental characteristic: after training, DBSCAN cannot assign a cluster to a new instance, only determine whether or not this instance is an anomaly. 

One solution is to train a KNeighborsClassifier on the clustering result, and use it to classify new instances:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(dbscan.components_, dbscan.labels_[dbscan.core_sample_indices_])

>>> X_new = np.array([[-0.5, 0], [0, 0.5], [1, -0.1], [2, 1]])
>>> knn.predict(X_new)
array([1, 0, 1, 0])

To determine whether a new instance is an anomaly, use the kneighbors() method to find its k nearest neighbors (in the training set) and count how many are anomalies (label = –1). If most of them are anomalies, then the new instance is probably also an anomaly. 

**Figure 9-15. Decision boundary between two clusters**   
![Figure9-15.jpg](./09.Chapter-09/Figure9-15.jpg) 

DBSCAN is a remarkably simple yet powerful algorithm, capable of identifying any number of clusters, of any shape. It is robust to outliers, and it has just two hyperparameters (eps and min_samples). However, if the density varies significantly across the clusters, it can be impossible for it to capture all the clusters properly. Its computational complexity is roughly O(m log m), making it pretty close to linear with regard to the number of instances. However, Scikit-Learn's implementation can require up to O(m²) memory if eps is large. 

---

### **Other Clustering Algorithms** 

Scikit-Learn implements several more clustering algorithms. Here's a brief overview: 

**Agglomerative clustering** - A hierarchy of clusters is built from bottom to top. Think of many tiny bubbles floating on water and gradually attaching to each other until there's just a single big group of bubbles. Similarly, at each iteration agglomerative clustering connects the nearest pair of clusters (starting with individual instances). If you draw a tree with a branch for every pair of clusters that merged, you get a binary tree where every node represents a cluster. This approach scales very well to large numbers of instances or clusters, and it can capture clusters of various shapes. It can scale to large numbers of instances if you provide a connectivity matrix. This is a sparse m × m matrix that indicates which pairs of instances are neighbors (e.g., returned by sklearn.neighbors.kneighbors_graph()). Without a connectivity matrix, the algorithm does not scale well to large datasets. 

**BIRCH** - The Balanced Iterative Reducing and Clustering using Hierarchies algorithm was designed specifically for very large datasets, and it can be faster than batch K-Means, with comparable results, as long as the number of features is not too large (<20). During training, it builds a tree structure containing just enough information to quickly assign each new instance to a cluster, without having to store all instances in the tree. The tree structure is built incrementally, ensuring that outliers have as little impact as possible on the final result. 

**Mean-Shift** - This algorithm starts by placing a circle centered on each instance, then for each circle it computes the mean of all the instances located within it, and it shifts the circle so that it is centered on the mean. Next, it iterates this mean-shifting step until all the circles stop moving (i.e., until each of them is centered on the mean of the instances it contains). This algorithm shifts the circles in the direction of higher density, until each of them has found a local density maximum. Finally, all the circles that have settled in (or near) the same local maximum are assigned to the same cluster. Mean-Shift has two important parameters: the circle's radius and the algorithm to use to find the neighbors in each circle. Mean-Shift has some of the same features as DBSCAN, in particular it can find any number of clusters, of any shape, it relies on local estimates of the density, and it can also detect anomalies. However, it tends to chop large clusters into smaller ones. 

**Affinity propagation** - This algorithm uses a voting system, where instances vote for similar instances and once the algorithm converges, each representative and its voters form a cluster. Affinity propagation can detect any number of clusters of different sizes. Unfortunately, this algorithm has a computational complexity of O(m²), so it is not suited for large datasets. 

**Spectral clustering** - This algorithm takes a similarity matrix between the instances and creates a low-dimensional embedding from it (i.e., it reduces its dimensionality), then it uses another clustering algorithm in this low-dimensional space (Scikit-Learn's implementation uses K-Means). Spectral clustering can capture complex cluster structures, and it can also be used to cut graphs (e.g., to identify clusters of friends on a social network). It does not scale well to large numbers of instances, and it does not behave well when the clusters have very different sizes. 

---

## **Gaussian Mixtures** 

A **Gaussian mixture model (GMM)** is a probabilistic model that assumes that the instances were generated from a mixture of several Gaussian distributions whose parameters are unknown. All the instances generated from a single Gaussian distribution form a cluster that typically looks like an ellipsoid. Each cluster can have a different ellipsoidal shape, size, density, and orientation, just like in Figure 9-11 (right). When you observe an instance, you know it was generated from one of the Gaussian distributions, but you are not told which one, and you do not know what the parameters of these distributions are. 

**Figure 9-16. A graphical representation of a Gaussian mixture model**   
![Figure9-16.jpg](./09.Chapter-09/Figure9-16.jpg) 

There are several GMM variants. In the simplest variant, implemented in the GaussianMixture class, you must know in advance the number k of Gaussian distributions. The dataset X is assumed to have been generated through the following probabilistic process: for each instance, a cluster is picked randomly among k clusters. The probability of choosing the jth cluster is defined by the cluster's weight, φʲ. The index of the cluster chosen for the ith instance is noted z⁽ⁱ⁾. If z⁽ⁱ⁾ = j, meaning the ith instance has been assigned to the jth cluster, the location x⁽ⁱ⁾ of this instance is sampled randomly from the Gaussian distribution with mean μʲ and covariance matrix Σʲ. This is noted x⁽ⁱ⁾ ~ N(μʲ, Σʲ). 

Let's train a Gaussian mixture model on the previous dataset:

In [ ]:
from sklearn.mixture import GaussianMixture

gm = GaussianMixture(n_components=3, n_init=10)
gm.fit(X)

Let's look at the parameters that the algorithm estimated:

In [ ]:
>>> gm.weights_
array([0.20965228, 0.4000662 , 0.39028152])
>>> gm.means_
array([[ 3.39909717,  1.05933727],
       [-1.40763984,  1.42710194],
       [ 0.05135313,  0.07524095]])
>>> gm.covariances_
array([[[ 1.14807234,  0.12750549],
        [ 0.12750549,  0.10011181]],
       [[ 0.29835504, -0.02313693],
        [-0.02313693,  0.95169231]],
       [[ 0.62354447, -0.02995475],
        [-0.02995475,  0.67106658]]])

Great, the algorithm indeed found three clusters. You can check that these are pretty close to the parameters used to generate the dataset. You can now use the model to predict which cluster each instance belongs to (hard clustering) and what the probability of each cluster is for each instance (soft clustering):

In [ ]:
>>> gm.predict(X)
array([2, 2, 1, ..., 0, 0, 0])
>>> gm.predict_proba(X)
array([[2.32389467e-02, 6.77397850e-07, 9.76760376e-01],
       [1.64685609e-02, 6.75361303e-04, 9.82856078e-01],
       ...])

This is a **generative model**, meaning you can sample new instances from it (note that they are ordered by cluster index):

In [ ]:
>>> X_new, y_new = gm.sample(6)
>>> X_new
array([[-0.43680083,  0.56926729],
       [ 0.52357523,  0.19085822],
       [-1.13605526,  1.37351048],
       [ 0.27743353,  0.23497545],
       [ 3.58850593,  0.82968176],
       [ 0.91465269,  1.11514819]])
>>> y_new
array([2, 2, 1, 2, 0, 2])

It is also possible to estimate the density of the model at any given location. This is achieved using the score_samples() method: for each instance it is given, this method estimates the log of the **probability density function (PDF)** at that location. The greater the score, the higher the density: 

**Figure 9-17. Cluster means, decision boundaries, and density contours of a trained Gaussian mixture model**   
![Figure9-17.jpg](./09.Chapter-09/Figure9-17.jpg) 

The figure shows the cluster means, the decision boundaries (dashed lines), and the density contours of this model. You can set the covariance_type hyperparameter to constrain the range of covariance matrices the algorithm can learn. The value "full" (the default) means that each cluster can take on any ellipsoidal shape of any size (it has its own covariance matrix). If you set it to "tied", then all clusters must have the same ellipsoidal shape (but they can be of different sizes in different dimensions, meaning they can be stretched and squeezed in various directions, but all in the same way). If you set it to "spherical" or "diag", then the clusters must be spherical, but they can have different diameters (spherical) or different dimensions in every dimension (diag). 

**Figure 9-18. Gaussian mixtures for tied clusters (left) and spherical clusters (right)**   
![Figure9-18.jpg](./09.Chapter-09/Figure9-18.jpg) 

---

### **Anomaly Detection Using Gaussian Mixtures** 

You can use Gaussian mixtures for anomaly detection: any instance located in a low-density region can be considered an anomaly. You must define what density threshold you want to use. For example, in a manufacturing company that tries to detect defective products, the ratio of defective products is usually well-known. Say it is equal to 4%, then you can set the density threshold to be the value that results in having 4% of the instances located in areas below that threshold density:

In [ ]:
densities = gm.score_samples(X)
density_threshold = np.percentile(densities, 4)
anomalies = X[densities < density_threshold]

**Figure 9-19. Anomaly detection using a Gaussian mixture model**   
![Figure9-19.jpg](./09.Chapter-09/Figure9-19.jpg) 

A downside of this approach is that it will not work well when the density of the "normal" instances is not significantly higher than the density of the anomalies. 

---

### **Selecting the Number of Clusters** 

With K-Means, you can use the inertia or the silhouette score to select the appropriate number of clusters. But with Gaussian mixtures, it is not possible to use these metrics because they are not reliable when the clusters are not spherical or have different sizes. Instead, you can try to find the model that minimizes a theoretical information criterion such as the **Bayesian information criterion (BIC)** or the **Akaike information criterion (AIC)**. 

**Equation 9-1. Bayesian information criterion (BIC) and Akaike information criterion (AIC)**   
![Eq9-1.jpg](./09.Chapter-09/Eq9-1.jpg) 

In these equations:
- m is the number of instances
- p is the number of parameters learned by the model
- L̂ is the maximized value of the likelihood function of the model

Both BIC and AIC penalize models that have more parameters to learn (e.g., more clusters) and reward models that fit the data well (this is why the formula contains –2 log(L̂)). They often end up selecting the same model. When they differ, the model selected by the BIC tends to be simpler (fewer parameters) than the one selected by the AIC, but tends to not fit the data quite as well (especially for larger datasets).

In [ ]:
>>> gm.bic(X)
8189.74345832983
>>> gm.aic(X)
8102.518178214792

We can compute the BIC and the AIC for various numbers of clusters and select the model with the lowest BIC or AIC. 

**Figure 9-20. A model's parametric function and derived functions**   
![Figure9-20.jpg](./09.Chapter-09/Figure9-20.jpg) 

**Figure 9-21. AIC and BIC for different numbers of clusters k**   
![Figure9-21.jpg](./09.Chapter-09/Figure9-21.jpg) 

In this figure, we can see that both BIC and AIC are lowest for k=3, so it is most likely the optimal number of clusters. Notice that k=4 also looks pretty good according to the AIC. In general, there is often some uncertainty regarding the optimal number of clusters. 

---

### **Bayesian Gaussian Mixture Models** 

Rather than manually searching for the optimal number of clusters, you can use the BayesianGaussianMixture class which is capable of giving weights equal (or close) to zero to unnecessary clusters. Just set the number of clusters n_components to a value that you have good reason to believe is greater than the optimal number of clusters (this assumes some minimal knowledge about the problem at hand), and the algorithm will eliminate the unnecessary clusters automatically. For example, let's set the number of clusters to 10 and see what happens:

In [ ]:
>>> from sklearn.mixture import BayesianGaussianMixture
>>> bgm = BayesianGaussianMixture(n_components=10, n_init=10)
>>> bgm.fit(X)
>>> np.round(bgm.weights_, 2)
array([0.4 , 0.21, 0.4 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])

Perfect: the algorithm automatically detected that only 3 clusters are needed, and the rest are just noise. 

**Figure 9-22. Bayesian Gaussian mixture model**   
![Figure9-22.jpg](./09.Chapter-09/Figure9-22.jpg) 

In this Bayesian GMM example, we set n_components=10, and yet the algorithm detected only 3 clusters. 

The concentration_prior hyperparameter determines how concentrated the model's prior is. The lower this value, the more the model will try to set weights equal to zero, and therefore the more clusters it will eliminate. The higher this value, the more clusters the model will keep active. 

**Figure 9-23. Using different concentration priors results in different numbers of clusters**   
![Figure9-23.jpg](./09.Chapter-09/Figure9-23.jpg) 

The BayesianGaussianMixture class relies on variational inference. Specifically, it uses a technique called **variational Bayesian inference**, where the goal is to approximate Bayesian inference—in particular, to estimate p(z|X), the probability distribution of the latent variables z given the observed variables X. 

**Equation 9-2. Bayes' theorem**   
![Eq9-2.jpg](./09.Chapter-09/Eq9-2.jpg) 

Unfortunately, in a GMM, you cannot compute p(z|X) because you can only compute p(X|z), and although it is theoretically possible to compute p(X) by integrating over all possible values of z, this is intractable for Gaussian mixtures (and for most other problems, except for trivial problems). 

**Equation 9-3. The evidence p(X) is often intractable**   
![Eq9-3.jpg](./09.Chapter-09/Eq9-3.jpg) 

So instead, variational inference tries to find a function q(z) that approximates p(z|X) and is also computationally tractable. To do this, it tries to find the function q(z) that minimizes the KL divergence from q(z) to p(z|X). 

**Equation 9-4. KL divergence from q(z) to p(z|X)**   
![Eq9-4.jpg](./09.Chapter-09/Eq9-4.jpg) 

By choosing an appropriate family of functions for q(z), with a manageable number of parameters to optimize (e.g., Gaussian distributions, determined entirely by their mean and covariance matrix), and by tweaking the math a bit, you end up with a function that you can actually optimize. 

So, how does a BGM eliminate unnecessary clusters? If the algorithm detects that a cluster does not have a sufficient number of instances supporting it (or has insufficient evidence), then it reduces the cluster's weight (by increasing its uncertainty, making its parameters less reliable). Similarly, if the data provides strong evidence for a cluster, the algorithm increases the cluster's weight. 

**Figure 9-24. Fitting a Gaussian mixture to nonellipsoidal clusters**   
![Figure9-24.jpg](./09.Chapter-09/Figure9-24.jpg) 

When a Gaussian mixture model is applied to nonellipsoidal data, such as the moons dataset, the algorithm ends up finding many small ellipsoidal clusters instead of the two moons. It takes multiple clusters to capture complex non-Gaussian patterns. 

---

## **Other Algorithms for Anomaly and Novelty Detection** 

Scikit-Learn implements a few other algorithms that are dedicated to anomaly or novelty detection: 

**PCA (and other dimensionality reduction techniques)** - If you compare the reconstruction error of a normal instance with the reconstruction error of an anomaly, the latter will generally be much larger. This is a fairly simple and often quite effective anomaly detection approach. 

**Fast-MCD (Minimum Covariance Determinant)** - Implemented by the EllipticEnvelope class, this algorithm is useful for outlier detection, in particular to clean up a dataset. It assumes that the normal instances (inliers) are generated from a single Gaussian distribution (not a mixture), and it also assumes that the dataset is contaminated with outliers that were not generated from this Gaussian distribution. When the algorithm estimates the parameters of the Gaussian distribution, it is careful to ignore the instances that are most likely outliers. This gives a better estimation of the elliptic envelope around the normal instances. 

**Isolation Forest** - This is an efficient algorithm for anomaly detection, especially in high-dimensional datasets. The algorithm builds a Random Forest in which each Decision Tree is grown randomly: at each node, it picks a feature randomly, then it picks a random threshold value (between the min and max values) to split the dataset in two. The dataset gradually gets chopped into pieces this way, until all instances end up isolated from the other instances. Anomalies are usually far from other instances, so on average (across all the trees in the forest) they tend to get isolated in fewer steps than normal instances. 

**Local Outlier Factor (LOF)** - This algorithm compares the density of instances around a given instance to the density around its neighbors. An anomaly is often more isolated than its k nearest neighbors. 

**One-class SVM** - This algorithm is better suited for novelty detection. Recall that a kernelized SVM classifier separates two classes by first (implicitly) mapping all the instances to a high-dimensional space, then separating the two classes using a linear SVM classifier within that space. Since we only have one class of instances, the one-class SVM algorithm instead tries to separate the instances in high-dimensional space from the origin. In the original space, this corresponds to finding a small region that encompasses all the instances. If a new instance does not fall within this region, it is an anomaly. There are a few hyperparameters to tweak: the usual ones for a kernelized SVM, plus a margin hyperparameter that corresponds to the probability of a new instance being mistakenly considered as novel when it is in fact normal. It works great, especially with high-dimensional datasets, but like all SVMs it does not scale to large datasets.